In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.cluster import KMeans
import pandas as pd
import requests
import glob

In [ ]:
gdf = gpd.read_file("New-York-City-Official-Public-Restrooms-Map.kml") 


In [ ]:
gdf.plot()
plt.show()

In [ ]:
gdf2 = gpd.read_file("new-york-city-boroughs.geojson")

In [ ]:
gdf2.plot()
plt.show() 

In [ ]:
!pip install contextily

In [ ]:
import contextily as cx

In [ ]:
df_wm = gdf.to_crs(epsg=3857)

In [ ]:
ax = df_wm.plot(figsize=(8, 8), alpha=.4, edgecolor="b")
cx.add_basemap(ax)

In [ ]:
gdf

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
gdf

In [ ]:
gdf['Description']

In [ ]:
gdf3 = gpd.read_file("ZillowNeighborhoods-NY.shp")

In [ ]:
gdf3.head()

In [ ]:
neighborhoods = gdf3.loc[lambda x: x['City'] == 'New York']

In [ ]:
neighborhoods

In [ ]:
## use regex to target the info and create a new df with all t hat.
# turn the str into categories
# turn the columns into numbers that represent the categories
# we do that (not in Kmeans) with the spy planes
# might have to normalize the categories -- some have 10 options, some have 5, they should be on same scale

In [ ]:
## Joins
## bathroom = pointsdf
## neighborhoods = polydf
## value count neighborhood

In [ ]:
import re

In [ ]:
## below I used AI to help me with the Def code.

In [ ]:
# Sample 'Description' text
# Assuming your 'gdf' already contains the 'Description' column
description_example = """
Unique Name: Aqueduct Walk Zone 2 (Parks2073)<br>Location Type: Park<br>Operator: NYC Parks<br>Open: Year Round<br>Hours of Operation: 8am-4pm, Open later seasonally<br>Accessibility: Fully Accessible<br>Restroom Type: Multi-Stall W/M Restrooms<br>Changing Stations: <br>Additional Notes: <br>Website: <br>Latitude: 40.858411<br>Longitude: -73.907994<br>Status: Operational
"""

# Define the function to extract values from the 'Description' text
def extract_info(description):
    info = {}
    
    # Define regex patterns for each field
    patterns = {
        "Unique Name": r"Unique Name:\s*(.*?)<br>",
        "Location Type": r"Location Type:\s*(.*?)<br>",
        "Operator": r"Operator:\s*(.*?)<br>",
        "Open": r"Open:\s*(.*?)<br>",
        "Hours of Operation": r"Hours of Operation:\s*(.*?)<br>",
        "Accessibility": r"Accessibility:\s*(.*?)<br>",
        "Restroom Type": r"Restroom Type:\s*(.*?)<br>",
        "Latitude": r"Latitude:\s*([-\d.]+)<br>",
        "Longitude": r"Longitude:\s*([-\d.]+)<br>",
        "Status": r"Status:\s*(.*?)<br>",
    }
    
    # Loop through the patterns and extract the information
    for key, pattern in patterns.items():
        match = re.search(pattern, description)
        if match:
            info[key] = match.group(1)
        else:
            info[key] = None  # If no match, assign None (or you could assign an empty string, depending on your preference)
    
    return info

# Apply the function to the 'Description' column and create new columns
extracted_info = gdf['Description'].apply(extract_info)

# Expand the dictionary into new columns in the GeoDataFrame
gdf = pd.concat([gdf, extracted_info.apply(pd.Series)], axis=1)

# Now gdf will have new columns like "Unique Name", "Location Type", "Operator", etc.


In [ ]:
gdf

In [ ]:
gdf.drop('Description', axis=1, inplace=True)

In [ ]:
gdf

In [ ]:
gdf.drop('Status', axis=1, inplace=True)

In [ ]:
gdf

In [ ]:
gdf.drop("Unique Name", axis=1, inplace=True)

In [ ]:
gdf

In [ ]:
# Apply One-Hot Encoding to categorical columns (Nominal)
categorical_cols = ['Location Type', 'Operator', 'Restroom Type', 'Hours of Operation']

# Use pd.get_dummies to create binary columns
gdf_encoded = pd.get_dummies(gdf, columns=categorical_cols, drop_first=True)  # drop_first to avoid dummy variable trap

# Check the new DataFrame
gdf_encoded.head()


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Apply Label Encoding for Ordinal columns
ordinal_cols = ['Open', 'Accessibility']

# Initialize the encoder
label_encoder = LabelEncoder()

# Apply label encoding to each ordinal column
for col in ordinal_cols:
    gdf_encoded[col] = label_encoder.fit_transform(gdf_encoded[col])

# Check the encoded DataFrame
gdf_encoded.head()


In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalize the numeric columns (Latitude and Longitude are already numeric, but others will need normalization)
scaler = StandardScaler()

# List of columns to normalize (you can add more if needed)
numeric_cols = ['Latitude', 'Longitude', 'Open', 'Accessibility']

# Apply the scaler to the selected numeric columns
gdf_encoded[numeric_cols] = scaler.fit_transform(gdf_encoded[numeric_cols])

# Check the scaled DataFrame
gdf_encoded.head()


In [ ]:
from sklearn.cluster import KMeans

# Assuming you want to cluster based on the encoded features:
X = gdf_encoded.drop(['Latitude', 'Longitude'], axis=1)  # Drop latitude and longitude if you're not clustering based on location

# Initialize KMeans
kmeans = KMeans(n_clusters=3)  # Choose the number of clusters (adjust n_clusters as needed)

# Fit the model
kmeans.fit(X)

# Assign the cluster labels back to the original DataFrame
gdf_encoded['Cluster'] = kmeans.labels_

# Check the final DataFrame with clusters
gdf_encoded.head()


In [ ]:
# Drop any non-numeric columns (e.g., 'Unique Name', 'Description', 'Status')
X = gdf_encoded.select_dtypes(include=[float, int])  # Keep only numeric columns



In [ ]:
from sklearn.cluster import KMeans

# Initialize KMeans (adjust the number of clusters as needed)
kmeans = KMeans(n_clusters=5, random_state=42)  # Set the desired number of clusters

# Fit the model to the numeric-only data
kmeans.fit(X)

# Add the cluster labels to the DataFrame
gdf_encoded['Cluster'] = kmeans.labels_

# Check the result
gdf_encoded.head()

In [ ]:
# Ensure that all GeoDataFrames are in EPSG:3857 for compatibility with contextily basemap
df_wm = gdf_encoded.to_crs(epsg=3857)         # K-Means clustered data
neighborhoods_wm = neighborhoods.to_crs(epsg=3857)  # NYC neighborhoods
gdf2_wm = gdf2.to_crs(epsg=3857)      # Public bathrooms

# Create the plot and set the figure size
fig, ax = plt.subplots(figsize=(10, 10))

# Step 1: Plot NYC map (basemap)
df_wm.plot(ax=ax, alpha=0.4, edgecolor="blue")

# Step 2: Plot the neighborhoods polygons (boundary of each neighborhood)
neighborhoods_wm.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=1, label="Neighborhood Boundaries")

# Step 3: Plot the public bathroom locations (gdf2)
#gdf2_wm.plot(ax=ax, color='clear', markersize=20, label="Public Bathrooms")

# Step 4: Plot the K-Means clustered data
df_wm.plot(ax=ax, column='Cluster', cmap='viridis', legend=True, alpha=0.6, edgecolor='black', label="K-Means Clusters")

# Step 5: Add basemap from contextily
cx.add_basemap(ax, crs=df_wm.crs.to_string())

# Set plot title
ax.set_title('NYC Map with K-Means Clusters, Neighborhoods, and Public Bathrooms', fontsize=15)

# Add a legend for all layers
ax.legend(loc='upper left', fontsize=10)

# Show the plot
plt.show()


In [ ]:
from sklearn.cluster import KMeans

# Assuming your gdf (or gdf_encoded) is ready for clustering and you've already dropped unnecessary columns
# Perform KMeans clustering
kmeans = KMeans(n_clusters=3)  # You can adjust n_clusters as needed
gdf['Cluster'] = kmeans.fit_predict(gdf[['Latitude', 'Longitude']])  # Use the appropriate columns for clustering

# Check if the 'Cluster' column was created
print(gdf.columns)


In [ ]:
# Plot the neighborhoods with thicker red lines (adjust linewidth and color)
neighborhoods_wm.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2, label="Neighborhood Boundaries")


In [ ]:
#ax.set_xlim([-8245000, -8230000])  # Adjust to proper NYC bounds
#ax.set_ylim([4970000, 4975000]) 

In [ ]:
gdf_encoded

In [ ]:
gdf_sorted = gdf_encoded.sort_values(by="Cluster")

In [ ]:
gdf_sorted

In [ ]:
cluster_col = gdf_sorted.pop('Cluster')

# Insert 'Cluster' column at the front (index 0)
gdf_sorted.insert(0, 'Cluster', cluster_col)

# Check the new column order
gdf_sorted

In [ ]:
cluster_range = range(1, 11)  # Test clusters from 1 to 10 (adjust as needed)

# List to store the inertia (WCSS) for each K
inertia = []

# Loop through different cluster numbers
for k in cluster_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(gdf[['Latitude', 'Longitude']])  # Use appropriate columns (latitude, longitude)
    inertia.append(kmeans.inertia_)  # Append the inertia value for each k

# Plot the elbow curve
plt.figure(figsize=(8, 6))
plt.plot(cluster_range, inertia, marker='o', linestyle='-', color='b')
plt.title('Elbow Method for Optimal Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia (WCSS)')
plt.grid(True)
plt.show()

In [ ]:
gdf_sorted.to_csv("bathrooms_sorted.csv", encoding="UTF-8")

In [ ]:
gdf_encoded

In [ ]:
gdf_encoded.to_csv("5-cluster-bathrooms-nyc.csv", encoding="UTF-8")